# Running Tximport and DESeq2 to calculate differentially expressed genes between HER2+ and TNBC BC samples

## Review our pipeline
In the pipeline image, the red text indicates the names of the R packages we will use to perform the analysis, the yellow boxes indicate the data (raw or processed), and the grey indicates what we will do in each step. Note that we will use [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) to identify differentially expressed genes (DEGs). This is a highly cited package that uses a negative binomial distribution to calculate DEGs. However, there are multiple pipelines and programs to do this. [Knoweng](http://education.knoweng.org/sequenceng/) has a nice interface to explore other options, including the strengths and weaknesses of each.
![](https://github.com/davidnboone/test-image/blob/master/overview-image.jpg?raw=true "pipeline overview")

## To this point we have:
1. Extracted and calculated HER2 and TNBC status of each patient.
2. Extracted count and abundance data for each transcript from those samples.
3. Prepped the transcipt count and abundance data for Tximport.

## In this notebook we will:
1. Run Tximport, which will collapse count and abundance data to the gene level.
2. Calculate differentially expressed genes using DESeq2.

___
___
___

## We still need the proper R packages loaded. We will need to start by reloading them with the same code as in the previous Notebook.
Of course you can use the typical library() command to load each as well.


In [1]:
#you can very easily use "install.packages" or "biocLite" to install the packages and "library" to load them
#however, instead I am installing a package called pacman that will determine if a package is already installed
#if it is not it will install it and after will load it
#this method is convenient when sharing code that requires the use of others packages

if (!require("pacman")) install.packages("pacman")
pacman::p_load(R.utils, data.table, tximport, DESeq2, biomaRt, jsonlite, BiocParallel, ggplot2, gplots, RColorBrewer, devtools, pheatmap)


Loading required package: pacman


## Set the working directory to the workspace set up in the previous notebook.
 You can check to deterimine if you are in the correct directory by examining your working directory. If it is not the proper workspace then change to proper directory.


In [2]:
getwd()

[1] "/Users/dnb14/Documents/jupyter/tcga_brca_rnaseq"

In [3]:
#only need to run if you are not in the proper working directory.
base_dir <- "/Volumes/Oesterreich"
setwd(base_dir)

project_name <- "TCGA_Her2_TNBC_DEGs"
date <- "2019_02_06" #use the same date as notebook 1 and 2
setwd(project_name)


___
___
___

# 1. Collapse transcript-level data to gene level using [Tximport](https://bioconductor.org/packages/release/bioc/html/tximport.html).
If you remember from Notebook 2, DESeq2 calculates differential *gene* expression not *transcript* expression. There are packages that will identify differentially expressed exons or transcripts, however these are still largely unreliable because many isoforms share the same exons, making precise mapping difficult. Accordinly, we need to collapse the transcript counts to gene counts. Tximport is a package that can do this for us. I highly recommend reading the [manual](https://bioconductor.org/packages/release/bioc/html/tximport.html) before starting. 

Tximport requires:
1. counts or tpm data (or output from pseudoaligners Salmon or Kallisto)
2. knowledge file that maps each transcript to gene. 

In notebook 2, we wrangled the count and tpm data into the proper format and created the knowledge file. Now we will run Tximport.

___
___

## 1.1  Read in all of the appropriate  datafiles generated in notebook 1 and 2. 
If you compiled all notebooks into one, and you have the following in your global envirenment, 'brca.clin.selected', 'brca.untran.counts.df', 'brca.untran.tpm.df', 'date', and 'tx2gene81' you can skip the next block of code and go to 1.2 Run Tximport.

As explained above For tximport we need 
1. the unlogged transcript counts data
2. the unlogged transcript tpm data
3. tx2gene81 

After tximport, we will need the clinical data for DESeq2
4. brca.clin.selected

You can read them in using the code below. **Be sure the commands point to the appropriate directories that contain the data**


In [7]:
#read in the unlogged transcript count data. Because it was saved as an Rda we can use the 
#load() function and it will retain the appropriate variable name
load(file = file.path(date, "df.unlogcounts_her2_tnbc.Rda"))

#read in the unlogged transcript tpm data.
load(file = file.path(date, "df.unlogtpm_her2_tnbc.Rda"))

#read in the tx2gene file
load(file = file.path(date, "tx2gene81.Rda"))

#read in the clinical data
brca.clin.selected <- read.table(file = file.path(date,"brca.clin.data.her2.tnbc.txt"), header = TRUE, stringsAsFactors = FALSE)

___
___

## 1.2 Run Tximport
The resulting file of tximport will provide the **gene** count data. Again, please refer to the Tximport manual to see all the ways to run it. We are providing count and tpm data instead of the output data from a series of data from pseudoaligners, so this is slightly different than if you created your own counts files from fastq files using Salmon or Kallisto. Please, note that in this method, we create a list named txi. This list contains all the abundance data (tpm data that we provide), counts data (again from the counts dataframe generated in Notebook 2), and the length of each transcript. We only need to provide the length data if we are converting counts to tpm or vice versa, which we are not doing. Hence, we will just give dummy values of the same length. We also tell tximport that we are not calculating counts from abundance data for each value. To summarize to gene level, we then provide that list (txi) and the knowledge file tx2gene (generated in Notebook 2).

Before we run tximport, let's confirm that the transcript names in the counts dataframe and tx2gene match.


In [8]:
#examine transcript names in tx2gene
head(tx2gene81)

#examine transcript names in counts dataframe. Here we are looking at the first 5 rows and 5 columns
brca.untran.counts.df[1:5, 1:5]

transcript,gene
ENST00000456328,ENSG00000223972
ENST00000450305,ENSG00000223972
ENST00000488147,ENSG00000227232
ENST00000619216,ENSG00000278267
ENST00000473358,ENSG00000243485
ENST00000469289,ENSG00000243485


,TCGA.3C.AALI.01,TCGA.3C.AALK.01,TCGA.A1.A0SK.01,TCGA.A1.A0SM.01,TCGA.A1.A0SN.01
ENST00000548312.5,10.01273,8.991491,0.000000,8.237484,5.519675
ENST00000527779.1,30.87603,18.463510,44.311332,0.000000,37.473682
ENST00000454820.5,0.00000,0.000000,0.000000,0.000000,0.000000
ENST00000535093.1,0.00000,0.000000,15.978063,0.000000,0.000000
ENST00000346219.7,0.00000,0.000000,7.487855,10.401091,8.667213


**note** the transcript names in the counts dataframe contains the version information. This is the number proceeding the period. tx2gene does not contain this information. We can tell tximport to ignore transcript versions.

In [9]:
#now to run tximport
#we are only providing the counts and tpm data in the list, but we are inserting in dummy values for length as they are not used in this analysis
#countsFromAbundance flag is set to no because these counts were generated directly from Kallisto and not backwards from tpm data
#abundance is our untransformed transcript tpm data
#counts is our untransformed transcript counts data
txi <- list(abundance = brca.untran.tpm.df, counts = brca.untran.counts.df, length = brca.untran.counts.df, countsFromAbundance = "no")

#txi is the list we made above with our transcript abundance and counts data
#tx2gene81 is the annotation information to translate from transcript to gene
#tx2gene81 was made in Notebook2

df.txi <- summarizeToGene(txi, tx2gene81, ignoreTxVersion = TRUE)

#what structure is df.txi
str(df.txi)



summarizing abundance
summarizing counts
summarizing length


[1] "list"

In [11]:
str(df.txi)

List of 4
 $ abundance          :'data.frame':	49922 obs. of  338 variables:
  ..$ TCGA.3C.AALI.01: num [1:49922] 6.5153 0.0504 58.7344 33.5409 8.2979 ...
  ..$ TCGA.3C.AALK.01: num [1:49922] 31.2981 0.0717 32.3299 10.595 5.8009 ...
  ..$ TCGA.A1.A0SK.01: num [1:49922] 6.28e+01 -2.18e-08 7.86e+01 6.53 1.37e+01 ...
  ..$ TCGA.A1.A0SM.01: num [1:49922] 31.5708 0.0154 46.3888 9.7262 7.9667 ...
  ..$ TCGA.A1.A0SN.01: num [1:49922] 8.6704 0.0171 122.0497 14.3974 10.0153 ...
  ..$ TCGA.A1.A0SO.01: num [1:49922] 47.3317 0.0989 93.0631 8.8083 16.1083 ...
  ..$ TCGA.A1.A0SP.01: num [1:49922] 38.06 0.146 50.576 5.317 8.986 ...
  ..$ TCGA.A2.A04P.01: num [1:49922] 25.9347 0.0346 13.4764 3.6072 3.3361 ...
  ..$ TCGA.A2.A04Q.01: num [1:49922] 18.8883 0.0476 29.0192 8.5853 5.9036 ...
  ..$ TCGA.A2.A04T.01: num [1:49922] 34.5 10.4 60.6 15.3 15.6 ...
  ..$ TCGA.A2.A04W.01: num [1:49922] 30.4351 0.0195 48.5732 5.7227 3.833 ...
  ..$ TCGA.A2.A04X.01: num [1:49922] 9.4439 0.0358 50.0049 5.1335 2.3086 ...

as we can see with the str(df.txi) call, df.txi is a list of all gene-level counts and abundance data for **49922 genes**. Now we need to extract the gene level counts and tpm data from df.txi. We will do this and coerce each into a dataframe for later use.

In [13]:
#after tximport we need to create variables with the gene counts and tpm
brca.untran.gene.counts <- as.data.frame(df.txi$counts)
brca.untran.gene.tpm <- as.data.frame(df.txi$abundance)


#to view the gene counts data. note that the rownames start with ENSG instead of ENST
message("counts data")
head(brca.untran.gene.counts, nrows = 10)

message("tpm data")
head(brca.untran.gene.tpm, nrows = 10)

#save the data
save(brca.untran.gene.counts, file = file.path(date, "brca.untran.gene.counts.Rda"))
save(brca.untran.gene.tpm, file = file.path(date, "brca.untran.gene.tpm.Rda"))


counts data


,TCGA.3C.AALI.01,TCGA.3C.AALK.01,TCGA.A1.A0SK.01,TCGA.A1.A0SM.01,TCGA.A1.A0SN.01,TCGA.A1.A0SO.01,TCGA.A1.A0SP.01,TCGA.A2.A04P.01,TCGA.A2.A04Q.01,TCGA.A2.A04T.01,⋯,TCGA.OL.A66P.01,TCGA.OL.A6VO.01,TCGA.OL.A97C.01,TCGA.PE.A5DC.01,TCGA.PE.A5DD.01,TCGA.S3.AA10.01,TCGA.S3.AA14.01,TCGA.S3.AA15.01,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01
ENSG00000000003,450.655142,3020.109054,6486.0687,3641.6943,875.8434,5252.389068,4349.910381,3964.5744,2835.217596,4355.0541,⋯,645.52538,4536.742724,3691.4786,2400.522368,1371.52306,1661.2877,2090.7229,1186.40144,976.8824,364.1602
ENSG00000000005,2.000078,3.999903,0.0000,1.0000,1.0000,4.999912,6.000185,3.0000,3.999903,731.0001,⋯,11.00031,6.000219,3556.0146,5.000156,24.99974,0.0000,7.0000,37.99979,1.0000,36.0012
ENSG00000000419,1821.031845,1405.018826,3620.0691,2324.9518,5508.9806,4677.870576,2524.012514,873.9736,1888.007985,3316.0325,⋯,1597.98718,2047.962746,1358.0296,1512.008500,1189.99584,2402.0106,1015.0289,1221.00364,1163.9743,1328.0154
ENSG00000000457,3688.499306,1940.060272,1323.6190,1972.4032,2573.8937,1543.572332,1095.212755,1096.8912,2412.455125,3308.7035,⋯,710.14563,1259.116073,754.7742,1227.154984,2042.30419,549.9404,1087.0107,961.86779,1708.3689,1660.7872
ENSG00000000460,769.629818,743.029409,1769.5213,1138.3500,1300.2310,2156.624673,1349.009842,820.7999,1170.767933,2410.5476,⋯,528.78813,1924.489990,364.9363,1044.840011,1096.15716,725.6080,509.8673,1310.12223,856.8481,957.7056
ENSG00000000938,383.997866,385.992250,115.9989,689.0194,411.9968,191.001862,924.994900,381.0095,1708.974047,839.0132,⋯,685.98680,461.996608,185.0003,310.004006,478.99150,1183.9836,169.0000,955.01035,215.9941,186.0013


tpm data


,TCGA.3C.AALI.01,TCGA.3C.AALK.01,TCGA.A1.A0SK.01,TCGA.A1.A0SM.01,TCGA.A1.A0SN.01,TCGA.A1.A0SO.01,TCGA.A1.A0SP.01,TCGA.A2.A04P.01,TCGA.A2.A04Q.01,TCGA.A2.A04T.01,⋯,TCGA.OL.A66P.01,TCGA.OL.A6VO.01,TCGA.OL.A97C.01,TCGA.PE.A5DC.01,TCGA.PE.A5DD.01,TCGA.S3.AA10.01,TCGA.S3.AA14.01,TCGA.S3.AA15.01,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01
ENSG00000000003,6.51534946,31.29805585,6.280911e+01,31.57077798,8.67041684,47.33168141,38.059551,25.93473872,18.88830498,34.544688,⋯,4.4470885,45.5182870,67.884085,22.16971860,17.3165612,1.651789e+01,31.8898304,9.242428,7.05971952,4.9771771
ENSG00000000005,0.05044589,0.07170003,-2.178597e-08,0.01536661,0.01712719,0.09889547,0.146012,0.03461922,0.04761009,10.390269,⋯,0.1356351,0.1028231,115.969091,0.08478087,0.7063652,-2.178597e-08,0.1873907,0.689528,0.01311623,0.8968211
ENSG00000000419,58.73442028,32.32993541,7.859218e+01,46.38880189,122.04966013,93.06309574,50.576155,13.47643623,29.01918536,60.612913,⋯,25.2861948,45.2074857,56.744003,33.11791951,33.2652626,5.324058e+01,34.9271880,21.119278,19.93375070,42.4709247
ENSG00000000457,33.54091741,10.59498195,6.533141e+00,9.72619167,14.39741632,8.80833987,5.317215,3.60721561,8.58530095,15.318031,⋯,2.7659811,6.9569049,8.621612,6.30240497,15.0684562,3.374201e+00,9.6647475,4.437816,6.65371733,11.1486465
ENSG00000000460,8.29792941,5.80092100,1.373277e+01,7.96667122,10.01529016,16.10825740,8.985589,3.33609228,5.90359760,15.559711,⋯,2.1640055,15.1271171,5.323229,6.50888160,9.9350641,5.685957e+00,5.9254496,7.828398,4.01198994,10.2266756
ENSG00000000938,5.89488620,4.21810256,1.056610e+00,6.45857008,4.07883811,1.68323530,7.893180,2.47017262,11.13667583,6.771309,⋯,4.4954092,4.3927861,3.793558,3.13872136,5.7674670,1.177756e+01,2.7647520,7.713530,1.69585923,2.5440392


**note** the tpm data is lower than the counts data for each gene because it is a normalized value correcting for gene length and sequencing depth. [This](https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/) blog does a nice job explaining how the calculation is performed and compares it to two other common RNAseq normalization methods rpkm and fpkm.



## 1.3 Check the results of tximport
Let's also look at the difference between the output of tximport and the input counts file.
First let's identify the transcripts from the first gene listed in the counts file above, "ENSG00000000003".

In [14]:
gene1_transcripts <- as.vector(tx2gene81[tx2gene81$gene == "ENSG00000000003", 1])
gene1_transcripts

[1] "ENST00000612152" "ENST00000373020" "ENST00000614008" "ENST00000496771"
[5] "ENST00000494424"

Now let's look at the difference between the output and input of tximport in relation to 'ENSG00000000003'

In [16]:
message("gene level counts")
brca.untran.gene.counts["ENSG00000000003", ]

message("transcript level counts")
brca.untran.counts.df[gene1_transcripts, ]

gene level counts


,TCGA.3C.AALI.01,TCGA.3C.AALK.01,TCGA.A1.A0SK.01,TCGA.A1.A0SM.01,TCGA.A1.A0SN.01,TCGA.A1.A0SO.01,TCGA.A1.A0SP.01,TCGA.A2.A04P.01,TCGA.A2.A04Q.01,TCGA.A2.A04T.01,⋯,TCGA.OL.A66P.01,TCGA.OL.A6VO.01,TCGA.OL.A97C.01,TCGA.PE.A5DC.01,TCGA.PE.A5DD.01,TCGA.S3.AA10.01,TCGA.S3.AA14.01,TCGA.S3.AA15.01,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01
ENSG00000000003,450.6551,3020.109,6486.069,3641.694,875.8434,5252.389,4349.91,3964.574,2835.218,4355.054,⋯,645.5254,4536.743,3691.479,2400.522,1371.523,1661.288,2090.723,1186.401,976.8824,364.1602


transcript level counts


,TCGA.3C.AALI.01,TCGA.3C.AALK.01,TCGA.A1.A0SK.01,TCGA.A1.A0SM.01,TCGA.A1.A0SN.01,TCGA.A1.A0SO.01,TCGA.A1.A0SP.01,TCGA.A2.A04P.01,TCGA.A2.A04Q.01,TCGA.A2.A04T.01,⋯,TCGA.OL.A66P.01,TCGA.OL.A6VO.01,TCGA.OL.A97C.01,TCGA.PE.A5DC.01,TCGA.PE.A5DD.01,TCGA.S3.AA10.01,TCGA.S3.AA14.01,TCGA.S3.AA15.01,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01
ENST00000612152.4,30.825251,161.60187,752.205644,198.68747,57.258635,128.723856,270.291493,116.474189,158.421235,324.60973,⋯,49.202942,133.093919,284.13474,320.840037,77.20560,119.01742,216.89662,51.95151,127.417677,40.24936
ENST00000373020.8,411.028408,2805.97403,5422.468177,3403.90776,792.727511,5047.363228,4028.541566,3845.418434,2665.996100,4000.70749,⋯,594.558741,4306.992432,3369.62403,2050.125426,1238.29162,1506.45581,1820.61202,1113.36026,840.138756,323.91082
ENST00000614008.4,0.000000,0.00000,9.074246,17.51086,6.543322,9.526107,4.208977,0.000000,5.293227,0.00000,⋯,0.000000,8.411339,0.00000,5.367822,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000
ENST00000496771.5,8.801483,38.16790,302.320669,21.58824,19.313966,66.775878,46.868344,2.681771,5.507034,29.73691,⋯,1.763698,88.245035,37.71981,22.231332,49.54863,35.81444,53.21424,21.08967,9.325931,0.00000
ENST00000494424.1,0.000000,14.36526,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,⋯,0.000000,0.000000,0.00000,1.957750,6.47721,0.00000,0.00000,0.00000,0.000000,0.00000


You can see that the resulting gene level count data is essentially the sum of all the transcripts. This confirms that tximport worked (at least for this gene).

# 2. Calculate differential expressed genes using [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html)
DESeq2 is a powerful tool that performs differential gene expression analysis based on the negative binomial distribution, using normalized count data. Again, I highly recommend reading the [manual](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) before performing the next steps. There is even a [coursera lecture discussing DESeq2](https://www.coursera.org/lecture/bioconductor/count-based-rna-seq-analysis-XVfPX) that you might want to watch. 




To run DESeq2 we need to specify:
1. A dataframe explaining all samples (TCGA IDs) and their perscribed "group." For ex. TCGA.XX.XXXY.01 is TNBC and TCGA.XX.XXXX.01 is HER2.
2. The counts data with TCGA identifiers that are exactly the same as the dataframe above.
3. The comparison. This is easy for this example because there are only 2 groups and we are calculating the DE genes between those groups.



## 2.1 Before creating the three items needed, let's check that the TCGA identifiers are exactly the same between the counts and clinical data.

In [17]:
#we need to make sure that TCGA ids are the same in the clinical and counts data
head(brca.clin.selected)
head(brca.untran.gene.counts)



sampleID,breast_carcinoma_estrogen_receptor_status,breast_carcinoma_progesterone_receptor_status,lab_proc_her2_neu_immunohistochemistry_receptor_status,lab_procedure_her2_neu_in_situ_hybrid_outcome_type,HER2,TNBC
TCGA-3C-AAAU-01,Positive,Positive,Negative,unknown,Negative,NOT_TNBC
TCGA-3C-AALI-01,Positive,Positive,Positive,unknown,Positive,NOT_TNBC
TCGA-3C-AALJ-01,Positive,Positive,Indeterminate,unknown,unknown,NOT_TNBC
TCGA-3C-AALK-01,Positive,Positive,Positive,unknown,Positive,NOT_TNBC
TCGA-4H-AAAK-01,Positive,Positive,Equivocal,unknown,unknown,NOT_TNBC
TCGA-5L-AAT0-01,Positive,Positive,Negative,unknown,Negative,NOT_TNBC


,TCGA.3C.AALI.01,TCGA.3C.AALK.01,TCGA.A1.A0SK.01,TCGA.A1.A0SM.01,TCGA.A1.A0SN.01,TCGA.A1.A0SO.01,TCGA.A1.A0SP.01,TCGA.A2.A04P.01,TCGA.A2.A04Q.01,TCGA.A2.A04T.01,⋯,TCGA.OL.A66P.01,TCGA.OL.A6VO.01,TCGA.OL.A97C.01,TCGA.PE.A5DC.01,TCGA.PE.A5DD.01,TCGA.S3.AA10.01,TCGA.S3.AA14.01,TCGA.S3.AA15.01,TCGA.UL.AAZ6.01,TCGA.UU.A93S.01
ENSG00000000003,450.655142,3020.109054,6486.0687,3641.6943,875.8434,5252.389068,4349.910381,3964.5744,2835.217596,4355.0541,⋯,645.52538,4536.742724,3691.4786,2400.522368,1371.52306,1661.2877,2090.7229,1186.40144,976.8824,364.1602
ENSG00000000005,2.000078,3.999903,0.0000,1.0000,1.0000,4.999912,6.000185,3.0000,3.999903,731.0001,⋯,11.00031,6.000219,3556.0146,5.000156,24.99974,0.0000,7.0000,37.99979,1.0000,36.0012
ENSG00000000419,1821.031845,1405.018826,3620.0691,2324.9518,5508.9806,4677.870576,2524.012514,873.9736,1888.007985,3316.0325,⋯,1597.98718,2047.962746,1358.0296,1512.008500,1189.99584,2402.0106,1015.0289,1221.00364,1163.9743,1328.0154
ENSG00000000457,3688.499306,1940.060272,1323.6190,1972.4032,2573.8937,1543.572332,1095.212755,1096.8912,2412.455125,3308.7035,⋯,710.14563,1259.116073,754.7742,1227.154984,2042.30419,549.9404,1087.0107,961.86779,1708.3689,1660.7872
ENSG00000000460,769.629818,743.029409,1769.5213,1138.3500,1300.2310,2156.624673,1349.009842,820.7999,1170.767933,2410.5476,⋯,528.78813,1924.489990,364.9363,1044.840011,1096.15716,725.6080,509.8673,1310.12223,856.8481,957.7056
ENSG00000000938,383.997866,385.992250,115.9989,689.0194,411.9968,191.001862,924.994900,381.0095,1708.974047,839.0132,⋯,685.98680,461.996608,185.0003,310.004006,478.99150,1183.9836,169.0000,955.01035,215.9941,186.0013


**Note** that the IDs are slightly different. In the clinical data the IDs contain "-" while the counts data IDs contain periods. We need to correct this in our dataframe explaining our samples. We can do that by simply subbing "." for "-" in the clinical data (to match the counts data) using gsub with the identifiers of interest. For us, the IDs of interest are HER2 and TNBC samples that also have counts data. We do this with the blocks of code below.

In [18]:
#note that the IDs are slightly different. In the clinical data the IDs contain "-" while in the counts data IDs contain "."
#we will need to rectify when running DESeq2

#make vectors of all HER2 positive and TNBC IDs from the clinical data 
#and change - to . to match brca.untran.gene.counts (output of tximport) 
her2.ids <- as.vector(gsub("-", ".", brca.clin.selected[brca.clin.selected$HER2 == "Positive", 1]))
tnbc.ids <- as.vector(gsub("-", ".", brca.clin.selected[brca.clin.selected$TNBC == "TNBC", 1]))

#examine the results of the gsub to confirm it worked

#old clinical data names
message("old clinical data names")
brca.clin.selected[1:5 ,1]

#converted her2 IDs
message("converted  HER2 IDs")
her2.ids[1:5]

#list of IDs from counts data
message("IDs from counts data")
colnames(brca.untran.gene.counts)[1:5]




old clinical data names


[1] "TCGA-3C-AAAU-01" "TCGA-3C-AALI-01" "TCGA-3C-AALJ-01" "TCGA-3C-AALK-01"
[5] "TCGA-4H-AAAK-01"

converted  HER2 IDs


[1] "TCGA.3C.AALI.01" "TCGA.3C.AALK.01" "TCGA.A1.A0SM.01" "TCGA.A1.A0SN.01"
[5] "TCGA.A2.A04W.01"

IDs from counts data


[1] "TCGA.3C.AALI.01" "TCGA.3C.AALK.01" "TCGA.A1.A0SK.01" "TCGA.A1.A0SM.01"
[5] "TCGA.A1.A0SN.01"

**Note** the IDs now match, but remember that not all HER2+ and TNBC samples have RNAseq data. If we intersect the column names from the counts data with the converted HER2+ and TNBC IDs, we will identify all identifiers with counts data. 

In [19]:
#it is possible to have clinical data without rnaseq data so we need to find patient samples that have both
her2.ids.final <- intersect(her2.ids, colnames(brca.untran.gene.counts))
tnbc.ids.final <- intersect(tnbc.ids, colnames(brca.untran.gene.counts))

message("number of HER2 samples")
length(her2.ids)

message("number of HER2 samples with counts data")
length(her2.ids.final)

message("number of TNBC samples")
length(tnbc.ids)

message("number of TNBC samples with counts data")
length(tnbc.ids.final)

message("number of HER2 and TNBC samples with counts")
length(her2.ids.final) + length(tnbc.ids.final)



number of HER2 samples


[1] 206

number of HER2 samples with counts data


[1] 181

number of TNBC samples


[1] 176

number of TNBC samples with counts data


[1] 157

number of HER2 and TNBC samples with counts


[1] 338

In [20]:
#we will need these IDs later, so save them to file
save(her2.ids.final, file = file.path(date, "her2.ids.final.Rda"))
save(tnbc.ids.final, file = file.path(date, "tnbc.ids.final.Rda"))


This is positive reaffirmation that everything is working. As in Notebook 2, we have 206 HER2+ IDs and 176 TNBC IDs. Also, the number of HER2 and TNBC IDs with counts data sums to 338, as it did in Notebook 2. Now we can create what is necessary to run DESeq2:

1. A dataframe explaining all samples (TCGA IDs) and their perscribed "group." For ex. TCGA.XX.XXXY.01	is TNBC and TCGA.XX.XXXX.01 is HER2.
2. The counts data with TCGA identifiers that are exactly the same as the dataframe above. These values also must be integers to run DESeq2.

## 2.2 Create the counts dataframe for DESeq2
The counts dataframe is very straightforward to create now that the IDs match perfectly. We will subset the entire gene.counts data frame to create HER2 positive and TNBC-specific dataframes of the counts. We will then bind them together and floor because DESeq2 expects integers. You could also round the counts, it will not make a noticable difference.

In [21]:
######################################################################################
############### create counts dataframe for DESeq2 ###################################
######################################################################################

#now create counts dataframes of her2 and tnbc, bind together, and floor because they must be integers for DESeq2
her2.counts.df <- brca.untran.gene.counts[, her2.ids.final] 
tnbc.counts.df <- brca.untran.gene.counts[, tnbc.ids.final]

#bind these together and floor to make all values integers.
her2.tnbc.counts.df <- floor(cbind(her2.counts.df, tnbc.counts.df))

#examine the dataframe
head(her2.tnbc.counts.df)

#check the dimensions to check the correct number of samples and the number of genes
message("dimensions of counts data frame")
dim(her2.tnbc.counts.df)

,TCGA.3C.AALI.01,TCGA.3C.AALK.01,TCGA.A1.A0SM.01,TCGA.A1.A0SN.01,TCGA.A2.A04W.01,TCGA.A2.A04X.01,TCGA.A2.A0CX.01,TCGA.A2.A0D1.01,TCGA.A2.A0EY.01,TCGA.A2.A0SY.01,⋯,TCGA.LL.A740.01,TCGA.OL.A5D6.01,TCGA.OL.A5D7.01,TCGA.OL.A5RW.01,TCGA.OL.A66I.01,TCGA.OL.A66P.01,TCGA.OL.A6VO.01,TCGA.OL.A97C.01,TCGA.S3.AA10.01,TCGA.S3.AA15.01
ENSG00000000003,450,3020,3641,875,2779,1429,3276,3980,2004,2056,⋯,1489,1434,2034,1249,2389,645,4536,3691,1661,1186
ENSG00000000005,2,3,1,1,1,3,1,8,15,70,⋯,1,2,4,0,56,11,6,3556,0,37
ENSG00000000419,1821,1405,2324,5508,1935,3256,1924,1219,2143,1810,⋯,2093,1141,1880,1544,3579,1597,2047,1358,2402,1221
ENSG00000000457,3688,1940,1972,2573,1088,1336,2156,1231,2562,2314,⋯,1130,1342,1737,1282,1986,710,1259,754,549,961
ENSG00000000460,769,743,1138,1300,503,508,1157,751,1020,851,⋯,654,387,2625,1859,2529,528,1924,364,725,1310
ENSG00000000938,383,385,689,411,226,869,344,180,310,510,⋯,730,534,689,189,1277,685,461,185,1183,955


dimensions of counts data frame


[1] 49922   338

**note** if the dataframe is correct it should contain 338 samples (as calculated above HER2 positive + TNBC samples with counts data) and 49922 genes (determined by tximport output).

## 2.3 Create knowledge dataframe for DESeq2
The knowledge dataframe is simply a two column dataframe. Column one is the TCGA ID of each sample and column two is their corresponding breast cancer subtype; "HER2" or "TNBC." There are multiple ways to create the dataframe. We will take the simple approach and create a vector that will specify breast cancer subtype by repeating "HER2" by the number of HER2 samples and "TNBC" by the number of TNBC samples. We calculated those numbers in 2.1 above. We will then append that vector to a vector of sample names from the final counts dataframe. **note** it is critical that we join the HER2 and TNBC vectors the same way we joined the counts dataframes above (HER2 then TNBC). We should then check that this is correct. Finally, we also will specify each row name as the same sample name.

In [22]:
#########################################################################################
################### create knowledge file for DESeq2 see manual ##########################
##########################################################################################

numHER2 <- ncol(her2.counts.df)
numTNBC <- ncol(tnbc.counts.df)

#BC_subtype defines what columns are HER2 samples vs TNBC samples
BC_subtype <- c(rep("HER2", numHER2), rep("TNBC", numTNBC))

#samples of interest are merely the name of the samples that we will examine
samplesofinterest <- colnames(her2.tnbc.counts.df)

#sampleTable is the combination of the above two
#it is a dataframe with sample name and if it is HER2 or TNBC
sampleTable = data.frame(sample = samplesofinterest, BC_subtype = BC_subtype, row.names = samplesofinterest)

#examine the sampleTable
head(sampleTable)

,sample,BC_subtype
TCGA.3C.AALI.01,TCGA.3C.AALI.01,HER2
TCGA.3C.AALK.01,TCGA.3C.AALK.01,HER2
TCGA.A1.A0SM.01,TCGA.A1.A0SM.01,HER2
TCGA.A1.A0SN.01,TCGA.A1.A0SN.01,HER2
TCGA.A2.A04W.01,TCGA.A2.A04W.01,HER2
TCGA.A2.A04X.01,TCGA.A2.A04X.01,HER2


If the dataframe is in the correct format, you should see a dataframe with rownames that are the sample name and two columns -'sample' and'BC_subtype'. Let's confirm the samples are correctly labeled by making sure that the samples labled HER2 and TNBC in the sampleTable correspond to the HER2 and TNBC IDs we calculated above. 

In [23]:
#subset dataframe to HER2 positive samples and check they are in vector of HER2 IDs
#that we created from the clinical data
message("HER2 logic")
sampleTable[sampleTable$BC_subtype == "HER2", 1] %in% her2.ids.final

#do the same with TNBC IDs
message("TNBC logic")
sampleTable[sampleTable$BC_subtype == "TNBC", 1] %in% tnbc.ids.final



HER2 logic


[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[136] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[151] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[166] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[181] TRUE

TNBC logic


[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[136] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[151] TRUE TRUE TRUE TRUE TRUE TRUE TRUE

**note** If all samples are correctly labelled, the output from the preceeding code should all be TRUE 

## 2.4 Run DESeq2
There are two steps to run DESeq2. The first (2.4a) is to create the object that DESeq2 uses for the statistical tests. The object class used by the DESeq2 package to store the read counts and the intermediate estimated quantities during statistical analysis is the DESeqDataSet. One caveat is that the counts data must be in a matrix instead of a dataframe. The second step (2.4b) is performing the statistical analysis, which is the most time consuming step.

### 2.4a Creating the DESeqDataSet

In [24]:
#########################################################################################
################### Run the first step of DESeq2 ##########################
##########################################################################################


#her2.tnbc.counts.df is our final count matrix from her2 and tnbc samples only
#sample table defines the subtpe for each sample
#design in this case is simply HER2 vs TNBC, which is definied in BC_subtype of the sampleTable. This can be much more complicated depending on your samples
#simply put, the design sets up the reference group for the differential expression analysis
deseqdata <- DESeqDataSetFromMatrix(countData = as.matrix(her2.tnbc.counts.df), colData = sampleTable, design = ~ BC_subtype)



converting counts to integer mode


**note** Taken directly from the reference [manual](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#the-deseqdataset), "By default, R will choose a reference level for factors based on alphabetical order. Meaning, if you do not specify which level you want to compare against (e.g. which level represents the control group), the comparisons will be based on the alphabetical order of the levels. We can manually set the comparison group using the relevel function on the deseqdata. You should only change the factor levels of variables in the design before running the DESeq2 analysis, not during or afterward." 

Accordingly, we will set the HER2 group as the reference label explicitly with the code below to avoid confusion when interpreting the results.

In [25]:
#the relevel function sets HER2 as the reference level for fold change calculations
deseqdata$BC_subtype <- relevel(deseqdata$BC_subtype, ref = "HER2")

#now save deseqdata in case we ever need to start from here again
save(deseqdata, file = file.path(date,"her2vstnbc.deseq.Rda"))

### 2.4b Perform the differential expression analysis with DESeq2
This is the most time and computational consuming portion of this analysis. Before we begin, we will clear the global environment and RAM except for the necessary items.

In [26]:
# to view what is in the global environment before clearing
message("in global environment before rm")
ls()

#remove large elements that are not needed. note we need tissuesource, samplesofinterest, and brca.untran.gene.tpm.df
rm(brca.untran.counts.df, brca.untran.gene.counts, her2.counts.df, brca.untran.tpm.df, df.txi, tx2gene81, tnbc.counts.df, txi)
gc()

#to make sure it worked
message("in global environment after rm")
ls()

#to free up ram
gc()



in global environment before rm


[1] "BC_subtype"              "brca.clin.selected"     
 [3] "brca.untran.counts.df"   "brca.untran.gene.counts"
 [5] "brca.untran.gene.tpm"    "brca.untran.tpm.df"     
 [7] "date"                    "deseqdata"              
 [9] "df.txi"                  "gene1_transcripts"      
[11] "her2.counts.df"          "her2.ids"               
[13] "her2.ids.final"          "her2.tnbc.counts.df"    
[15] "numHER2"                 "numTNBC"                
[17] "samplesofinterest"       "sampleTable"            
[19] "tnbc.counts.df"          "tnbc.ids"               
[21] "tnbc.ids.final"          "tx2gene81"              
[23] "txi"

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,5318280,284.1,10272058,548.6,NA,10272058,548.6
Vcells,51950130,396.4,524288982,4000.1,16384,650032873,4959.4


in global environment after rm


[1] "BC_subtype"           "brca.clin.selected"   "brca.untran.gene.tpm"
 [4] "date"                 "deseqdata"            "gene1_transcripts"   
 [7] "her2.ids"             "her2.ids.final"       "her2.tnbc.counts.df" 
[10] "numHER2"              "numTNBC"              "samplesofinterest"   
[13] "sampleTable"          "tnbc.ids"             "tnbc.ids.final"

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,5318307,284.1,10272058,548.6,NA,10272058,548.6
Vcells,51950146,396.4,419431185,3200.1,16384,650032873,4959.4


### Now we are ready to perform the statistical analysis. For the purposes of this exercise we will remove genes that are expressed at a low level across many samples to drastically increase the speed of the analysis. However at the end of this notebook, I include the code  (all commented out) that I normally would run that allows DESeq2 to perform a statistical test across all genes instead of arbitrary filtering. 
Note that we use the SnowParam function to make the code run across multiple threads. The number of threads depends on your machine and if you want to dedicate it completely to this task. On a relatively new laptop this step should not take more than a few hours. However, you might want to run overnight.

The verbose output should read something like:
```
using pre-existing size factors
estimating dispersions
gene-wise dispersion estimates: 4 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 4 workers
-- replacing outliers and refitting for 502 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing
```


In [27]:
##################################################################################################
################ code to execute DEseq2 after removing lowly expressed genes######################
################ this is only done to decrease computational time.  ###############################
############### normally run the code above and would skip this     ###############################
###################################################################################################

#will make DESeq2 run across 4 threads. You can change the number if running on a machine/server with more or less than 4.
numCores <- 4
register(SnowParam(numCores))


#these commands remove genes with low counts
#the first number specifies the number of counts for a given gene necessary to include in the deseq analysis
#the second number tells how many samples need to have that mean counts
#so in this case only genes with at least 200 counts in 100 samples will be kept
#this is much higher than I would normally do
#these numbers were selected only to reduce computational time for this exercise
#Normally, filtering is not necessary before running
dds <- estimateSizeFactors(deseqdata)
idx <- rowSums(counts(dds, normalized=TRUE) >= 200 ) >= 100
dds <- dds[idx,]

#this is the DESeq execution command that will take ~1hr
dds <- DESeq(dds, parallel = TRUE)

#save after running to save computational time later
save(dds, file = file.path(date, "her2.tnbc.results.Rda"))



using pre-existing size factors
estimating dispersions
gene-wise dispersion estimates: 4 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 4 workers
-- replacing outliers and refitting for 502 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


### Create an output directory to save all tables and graphs and make the differential expression results readable to any user

In [28]:
#make an output director
outdir <- "DESeq_output"
dir.create(file.path(date, outdir), recursive = TRUE)


#the results function summarizes the deseq results
#anythign with an alpha -FDR- greater than 0.001 is not considered significant
#you can obviously change this threshold so that everything is included or to be
#more stringent
results.fdr.threshold <- results(dds, alpha = 0.001, parallel = TRUE)
save(results.fdr.threshold, file = file.path(date, outdir, "her2.tnbc.results.Rda"))

#make it a dataframe so that we can export it to a csv
results.fdr.threshold.df <- as.data.frame(results.fdr.threshold)

#reorder based on pvalue
results.ordered <- results.fdr.threshold.df[order(results.fdr.threshold.df$pvalue, na.last = TRUE),]

write.csv(results.ordered, file = file.path(date, outdir, "her2.tnbc.results.csv"))

#check what results file looks like
head(results.ordered)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ENSG00000141736,111881.653,-3.907757,0.1497027,-26.10345,3.331450e-150,4.932878e-146
ENSG00000161395,10474.829,-3.289347,0.1397486,-23.53760,1.681954e-122,1.245234e-118
ENSG00000171428,4752.450,-4.556832,0.2036122,-22.37996,6.170364e-111,3.045486e-107
ENSG00000105388,4442.615,-5.409955,0.2490188,-21.72509,1.188649e-104,4.400083e-101
ENSG00000108932,1935.616,-3.610246,0.1665687,-21.67421,3.593392e-104,1.064147e-100
ENSG00000172057,18939.110,-2.936965,0.1386755,-21.17869,1.501635e-99,3.705785e-96


### Print to screen simple summary statistics
In R, I save the summary below to file by using the sink command that is not compatible in Jupyter Notebooks



In [29]:
#print summary of deseq to screen. 
#in native R, save this output by uncommenting the sink commands


#sink(file.path(date, "DESeqsummary_HER2_TNBC.txt"))
#note this is a summary  of the results BEFORE making it a dataframe
message("summary of dds")
summary(results.fdr.threshold)
#sink()

#this is a summary of the dataframe
#notice the difference in the summary statistics
message("summary of data table")
summary(results.ordered)

summary of dds



out of 14807 with nonzero total read count
adjusted p-value < 0.001
LFC > 0 (up)       : 3996, 27%
LFC < 0 (down)     : 3375, 23%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 150)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



summary of data table


    baseMean        log2FoldChange         lfcSE              stat         
 Min.   :   149.6   Min.   :-6.80133   Min.   :0.02868   Min.   :-26.1034  
 1st Qu.:   774.4   1st Qu.:-0.24955   1st Qu.:0.06541   1st Qu.: -3.0945  
 Median :  1870.1   Median : 0.04124   Median :0.08384   Median :  0.5140  
 Mean   :  4510.8   Mean   : 0.03333   Mean   :0.09918   Mean   :  0.2575  
 3rd Qu.:  4034.2   3rd Qu.: 0.33513   3rd Qu.:0.11492   3rd Qu.:  3.7944  
 Max.   :789875.0   Max.   : 5.86999   Max.   :0.59752   Max.   : 17.9066  
     pvalue               padj         
 Min.   :0.0000000   Min.   :0.000000  
 1st Qu.:0.0000000   1st Qu.:0.000000  
 Median :0.0005185   Median :0.001037  
 Mean   :0.1226463   Mean   :0.136650  
 3rd Qu.:0.1027516   3rd Qu.:0.136999  
 Max.   :0.9998667   Max.   :0.999867  

**note** this provides  a lot of information. First, there are only 14,807 genes with non-zero counts. This means we filtered a lot out before the differential expressional analysis. This number will be different if you run the code at the end of this notebook that do not exclude genes expressed at low levels. This summary also tells us that there are 3996 genes significantly upregulated in TNBC and 3375 significantly downregulated (FDR <0.001). 

The summary stats of the data table also tell us a great deal. The log2FCs range from -6.8 to 5.87. The padj column tells us that at least a quarter of the genes have an FDR of <0.000001 meaning the HER2 and TNBC groups are very different.

# The differential expression statistical analysis is complete.
In the next notebook, we will annotate the results file and visually explore the data.


___
___
___


## Below is the code to execute the differential expression analysis without ignoring genes expressed at low levels. Without a time restriction, I recommend doing this because arbitrary cutoffs will lead to false negatives.

In [ ]:
########################################################################################
################### execute DESeq2 #####################################################
#########################################################################################

#Normally I would run everything below, but this will take ~4 hours on 4 threads on this server
#accordingly, run the next code set that will remove genes with low expression, which will decrease
#the computational time to ~1hr.


load(file = file.path(date,"her2vstnbc.deseq.Rda"))

#will run across 4 threads
numCores <- 4
register(SnowParam(numCores))

dds.total <- DESeq(deseqdata, parallel = TRUE)
save(dds.total, file = "her2.dds.total.Rda")
Sys.time()

fdr = .001
results.total <- results(dds.total, alpha = fdr, parallel = TRUE)
save(results.total, file = file.path(date, "her2.tnbc.total.results.Rda"))

#reorder based on pvalue
results.total.df <- as.data.frame(results.total)
results.total.ordered <- results.total.df[order(results.total.df$pvalue, na.last = TRUE),]

write.csv(results.total.ordered, file = file.path(date, outdir, "her2.tnbc.results.total.ordered.csv"))

